In [1]:
import torch
import pickle
import numpy as np
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher, expand_level3
from heterogt.utils.train import train_with_early_stopping, PHENO_ORDER
from heterogt.utils.seed import set_random_seed
from heterogt.model.model import HeteroGTFineTune

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
config = Namespace(
    dataset = "MIMIC-IV",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"],
    task_index = 4,  # index of the task to train
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]", "[CLS]"],
    # only for tf layer
    attn_mask_dicts = [{1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}, 
                       {1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 5:[6,7], 6:[1,2,3,4,5,6,7], 7:[1,2,3,4,5,6,7]}], 
    d_model = 64,
    num_heads = 4,
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,
    group_code_thre = 5,  # if there are group_code_thre diag codes belongs to the same group ICD code, then the group code is generated
    use_pretrained_model = True,
    pretrain_mask_rate = 0.7,
    pretrain_cls_ontology_weight = 5e-2,
    pretrain_visit_ontology_weight = 5e-2,
    pretrain_adm_type_weight = 0,
    dec_loss_lambda = 1e-2,
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: next_diag_12m


In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
group_code_sentences = [expand_level3()[1]]
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_sentences = [[str(c)] for c in set(ehr_full_data["AGE"].values.tolist())] # important of [[]]
adm_type_sentences = ehr_full_data["ADMISSION_TYPE"].values.tolist()
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
tokenizer = EHRTokenizer(age_sentences, group_code_sentences, diag_sentences, med_sentences, lab_sentences, 
                         pro_sentences, special_tokens=config.special_tokens, adm_types_sentences=adm_type_sentences)
config.label_vocab_size = len(PHENO_ORDER)  # a set of predifined diseases
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_vocab_size = tokenizer.token_number("age")
config.group_code_vocab_size = tokenizer.token_number("group")
print(f"Age vocabulary size: {config.age_vocab_size}")
print(f"Group code vocabulary size: {config.group_code_vocab_size}")

Age vocabulary size: 20
Group code vocabulary size: 19


In [8]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 3.327211839688884 %
Percentage of READMISSION in test dataset: 79.05368910014043 %
Percentage of STAY>7 days in test dataset: 34.87090850167441 %


In [9]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                 max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)

In [10]:
long_adm_seq_crite = 3
val_long_seq_idx, test_long_seq_idx = [], []
for i in range(len(val_dataset)):
    hadm_id = list(val_dataset.records.keys())[i]
    num_adms = len(val_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        val_long_seq_idx.append(i)
for i in range(len(test_dataset)):
    hadm_id = list(test_dataset.records.keys())[i]
    num_adms = len(test_dataset.records[hadm_id])
    if num_adms >= long_adm_seq_crite:
        test_long_seq_idx.append(i)
print(len(val_long_seq_idx), len(test_long_seq_idx))

857 876


In [11]:
num_group_code = []
for i in range(len(train_dataset)):
    input_ids, token_types, adm_index, age_ids, diag_group_codes, labels = train_dataset[i]
    count = (token_types[0] == 6).sum().item()
    num_group_code.append(count)
print("Mean group token numer per patient", np.mean(num_group_code))

Mean group token numer per patient 0.874014598540146


In [12]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [13]:
for batch in train_dataloader:
    pass  # just to check if the dataloader works
for batch in val_dataloader:
    pass  # just to check if the dataloader works
for batch in test_dataloader:
    pass  # just to check if the dataloader works
print("All pass!")

All pass!


In [14]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "f1"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "f1"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [15]:
input_ids, token_types, adm_index, age_ids, diag_code_group_dicts, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age IDs shape:", age_ids.shape)
print("Diag Code Group Dict number:", len(diag_code_group_dicts))
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 233])
Token Types shape: torch.Size([32, 233])
Admission Index shape: torch.Size([32, 233])
Age IDs shape: torch.Size([32, 6])
Diag Code Group Dict number: 32
Labels shape: torch.Size([32, 18])


# Model Walkthrough

In [16]:
# load pretrained model
if config.use_pretrained_model:
    pretrain_exp_name = (
    f"{config.dataset}-{config.pretrain_mask_rate}-{config.d_model}-{config.pretrain_cls_ontology_weight}-{config.pretrain_visit_ontology_weight}-{config.pretrain_adm_type_weight}"
)
    print(pretrain_exp_name)
    save_path = "./pretrained_models/" + pretrain_exp_name
    state_dict = torch.load(f"{save_path}/pretrained_model.pt", map_location="cpu")

MIMIC-IV-0.7-64-0.05-0.05-0


In [17]:
final_metrics, final_long_seq_metrics = [], []
for i in range(15):
    print(f"================================{i+1}==================================")
    model = HeteroGTFineTune(tokenizer=tokenizer, token_types=config.token_type, d_model=config.d_model, num_heads=config.num_heads, layer_types=['gnn', 'tf', 'gnn', 'tf'], max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, label_vocab_size=config.label_vocab_size, attn_mask_dicts=config.attn_mask_dicts,
                     use_cls_cat=True).to(device)
    if config.use_pretrained_model:
        model.load_weight(state_dict)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric, best_test_long_seq_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                                                            optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                                                            dec_loss_lambda=config.dec_loss_lambda, 
                                                                            val_long_seq_idx=val_long_seq_idx, test_long_seq_idx=test_long_seq_idx, 
                                                                            eval_metric=eval_metric, return_model=False)
    final_metrics.append(best_test_metric)
    final_long_seq_metrics.append(best_test_long_seq_metric)

================================1==================================


Running inference: 100%|██████████| 158/158 [00:07<00:00, 21.68it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          52.7273   5.0130   9.1555   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         78.4741  21.0373  33.1797   
Chronic kidney disease                       85.4651  63.1986  72.6644   
Chronic obstructive pulmonary disease         0.0000   0.0000   0.0000   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    84.2202  40.5477  54.7406   
Coronary atherosclerosis and related         79.1120  64.4207  71.0145   
Disorders of lipid metabolism                69.8630  70.0846  69.9736   
Essential hypertension                       72.8835  82.8490  77.5474   
Fluid and electrolyte disorders              57.5000   4.2540   7.9219   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 159/159 [00:07<00:00, 21.46it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          51.6949   5.2632   9.5536   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         72.6519  19.8191  31.1427   
Chronic kidney disease                       89.6963  65.2723  75.5596   
Chronic obstructive pulmonary disease         0.0000   0.0000   0.0000   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    83.8264  36.7329  51.0817   
Coronary atherosclerosis and related         78.1489  65.5725  71.3104   
Disorders of lipid metabolism                67.6564  69.5368  68.5837   
Essential hypertension                       71.5573  83.1505  76.9195   
Fluid and electrolyte disorders              58.2609   4.2218   7.8731   
Gastrointestinal hemorrhage           

...

                                           precision   recall       f1  \
Acute and unspecified renal failure          46.6042  17.1700  25.0946   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                  41.6667   2.6042   4.9020   
Cardiac dysrhythmias                         83.0128  58.5531  68.6699   
Chronic kidney disease                       83.5097  74.7435  78.8838   
Chronic obstructive pulmonary disease        64.2157  61.5023  62.8297   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    77.8426  69.2308  73.2845   
Coronary atherosclerosis and related         78.0291  77.3419  77.6840   
Disorders of lipid metabolism                74.4412  83.4877  78.7054   
Essential hypertension                       76.0431  80.8574  78.3764   
Fluid and electrolyte disorders              50.6494  17.2023  25.6820   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 158/158 [00:07<00:00, 21.49it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          44.0000  38.0294  40.7974   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                  20.0000   0.4975   0.9709   
Cardiac dysrhythmias                         78.5287  67.0562  72.3404   
Chronic kidney disease                       79.9440  73.6887  76.6890   
Chronic obstructive pulmonary disease        59.9532  57.1429  58.5143   
Conduction disorders                          0.0000   0.0000   0.0000   
Congestive heart failure; nonhypertensive    74.3075  68.7279  71.4089   
Coronary atherosclerosis and related         77.1605  82.1693  79.5862   
Disorders of lipid metabolism                76.1610  78.0127  77.0757   
Essential hypertension                       77.9215  77.5131  77.7167   
Fluid and electrolyte disorders              45.4001  37.4229  41.0274   
Gastrointestinal hemorrhage           

Running inference: 100%|██████████| 159/159 [00:07<00:00, 21.58it/s]


                                           precision   recall       f1  \
Acute and unspecified renal failure          45.1417  38.4814  41.5463   
Acute cerebrovascular disease                 0.0000   0.0000   0.0000   
Acute myocardial infarction                   0.0000   0.0000   0.0000   
Cardiac dysrhythmias                         75.6906  61.9442  68.1310   
Chronic kidney disease                       82.2124  73.3228  77.5136   
Chronic obstructive pulmonary disease        60.8899  61.0329  60.9613   
Conduction disorders                        100.0000   0.8772   1.7391   
Congestive heart failure; nonhypertensive    75.0680  71.5644  73.2743   
Coronary atherosclerosis and related         76.1314  83.5068  79.6487   
Disorders of lipid metabolism                75.5832  79.4550  77.4708   
Essential hypertension                       77.9535  78.6142  78.2825   
Fluid and electrolyte disorders              44.7101  38.8784  41.5908   
Gastrointestinal hemorrhage           

In [18]:
def topk_avg_performance_formatted(performances, long_seq_performances, k=5):
    metrics = ["f1", "auc", "prauc"]
    scores = {m: np.array([p[m] for p in performances]) for m in metrics}

    # 计算排名（值越大排名越靠前）
    ranks = {m: (-scores[m]).argsort().argsort() + 1 for m in metrics}
    avg_ranks = np.mean(np.stack([ranks[m] for m in metrics], axis=1), axis=1)

    # 选 top-k
    topk_idx = np.argsort(avg_ranks)[:k]
    final_avg = {m: np.mean([performances[i][m] for i in topk_idx]) for m in performances[0].keys()}
    final_std = {m: np.std([performances[i][m] for i in topk_idx], ddof=0) for m in performances[0].keys()}
    final_long_seq_avg = {m: np.mean([long_seq_performances[i][m] for i in topk_idx]) for m in long_seq_performances[0].keys()}
    final_long_seq_std = {m: np.std([long_seq_performances[i][m] for i in topk_idx], ddof=0) for m in long_seq_performances[0].keys()}

    # 打印结果（转百分比，均保留两位小数）
    print("Final Metrics:")
    for m in performances[0].keys():
        mean_val = final_avg[m]
        std_val = final_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")
    print("\nFinal Long Sequence Metrics:")
    for m in long_seq_performances[0].keys():
        mean_val = final_long_seq_avg[m]
        std_val = final_long_seq_std[m]
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")

In [19]:
topk_avg_performance_formatted(final_metrics, final_long_seq_metrics)

Final Metrics:
precision: 42.66 ± 1.17
recall: 38.34 ± 1.23
f1: 39.20 ± 0.49
auc: 78.71 ± 0.22
prauc: 44.67 ± 0.11

Final Long Sequence Metrics:
precision: 44.01 ± 2.27
recall: 41.24 ± 1.19
f1: 41.06 ± 0.48
auc: 78.85 ± 0.55
prauc: 47.47 ± 0.45
